In [1]:
%pip install gpt-2-simple

  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.2 MB/s eta 0:00:00
  Created wheel for gpt-2-simple: filename=gpt_2_simple-0.8.1-py3-none-any.whl size=24576 sha256=ee4baa98764b2abdc0432e877b958639de73485b19cbf9ebf5ce66a8d45bc99a
  Stored in directory: /root/.cache/pip/wheels/65/71/09/4761ea784b90a1ce6f1ab07c6eb553a35ce75bd23350bcf147
Successfully built gpt-2-simple
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 21.12.2 requires cupy-cuda115, which is not installed.
tfx-bsl 1.12.0 requires google-api-python-client<2,>=1.7.11, but you have google-api-python-client 2.83.0 which is incompatible.
tfx-bsl 1.12.0 requires pyarrow<7,>=6, but you h

# Import of the Dependecies

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import gpt_2_simple as gpt2
import pandas as pd
import numpy as np
import csv

# Dowloading the GPT-2 model, Seeding Numpy

In [3]:
model_name = "355M"
gpt2.download_gpt2(model_name=model_name)
np.random.seed(42)

Fetching checkpoint: 1.05Mit [00:00, 371Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 3.15Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 588Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:38, 36.7Mit/s]                                 
Fetching model.ckpt.index: 1.05Mit [00:00, 481Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 3.14Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 4.14Mit/s]


# Reading Reviews

In [4]:
reviews = pd.read_csv("/kaggle/input/imdb-movies-review/imdb.csv")

In [5]:
reviews

,text,rating,sentiments
0,I have just seen Caribe a couple of nights ago...,3,0
1,I always enjoy this movie when it shows up on ...,8,1
2,I suppose you could say this film has a grain ...,2,0
3,"This experimental silent film, made in Switzer...",4,0
4,"In The Book of Life, Martin Donovan plays Jesu...",10,1
...,...,...,...
49995,Even in her glasses wearing geek mode Kathy Ir...,3,0
49996,"Imagine Diane from Cheers, the self centered o...",1,0
49997,I have never seen such terrible performances i...,1,0
49998,"Not sure if this counts as a spoiler or not, s...",10,1


# Testing, Training Split With stratify

In [6]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(reviews['text'].tolist(), reviews['sentiments'].tolist(),
shuffle=True, test_size=0.1, random_state=1, stratify=reviews['sentiments'])

# Saving the testing split to a file for later evaluation 

In [7]:
len(X_test)
df = pd.DataFrame({'text':X_test, 'sentiments':y_test})
df.to_csv("testReview.csv")

In [8]:
testReview = pd.read_csv("/kaggle/working/testReview.csv")
testReview

,Unnamed: 0,text,sentiments
0,0,We have reached the ceiling of implausibility ...,0
1,1,"This was a crappy, miserably acted movie based...",0
2,2,One wonders how the script came to be written....,0
3,3,"Like the other guy said It sux , you can count...",0
4,4,A true wholesome American story about teenager...,1
...,...,...,...
4995,4995,"First of all, I apologize for my English. <br ...",0
4996,4996,They Made Me a Criminal is a remake of an earl...,1
4997,4997,I basically skimmed through the movie but just...,0
4998,4998,"For such films like `Anchors Aweigh', few have...",1


# Reading training Sample and saving as Encoder format

<p>The format is as follows:
REVIEW_ {VALUE} _SENTIMENTS_ {VALUE} _END_</p>

In [9]:
sentiment= "sentiments"
review_text ="text"
length=0
map_label = {0:'negative', 1: 'positive','0':'negative', '1':'positive'}
i=0

file_name = "processed_review.txt"
with open(file_name, mode='w') as open_file:
    for index,text in enumerate(X_train):
        if len(text.split())<460:
            if y_train[index] in map_label:
                str= "REVIEW_ {} _SENTIMENTS_ {} _END_".format(text, map_label[y_train[index]])
                open_file.write(str)
                length+=1
        i=i+1

print(length)

40689


# Encoding training sample for faster loading during training

In [10]:
model_name = "355M"
encoded = gpt2.encode_dataset('/kaggle/working/processed_review.txt', model_name=model_name)

Reading files


100%|██████████| 1/1 [00:46<00:00, 46.75s/it]


Writing text_encoded.npz


# Starting Gpt Session

In [11]:
sess = gpt2.start_tf_sess()

# Finetuning GPT-2 on the training sample

In [13]:
gpt2.finetune(sess,
              dataset="text_encoded.npz",
              model_name=model_name,
              steps=41000,
              restore_from='fresh',
              run_name='run1',
              print_every=100,
              sample_every=200,
              save_every=500,
              only_train_transformer_layers = True
              ,accumulate_gradients = 1
              )

For larger models, the recommended finetune() parameters are:
	use_memory_saving_gradients = True
	only_train_transformer_layers = True
	accumulate_gradients = 1

Loading checkpoint models/355M/model.ckpt
Loading dataset...


100%|██████████| 1/1 [00:00<00:00,  3.03it/s]


dataset has 10245534 tokens
Training...
[100 | 77.38] loss=3.12 avg=3.12
[200 | 144.37] loss=3.05 avg=3.08
======== SAMPLE 1 ========
 confusing. Even the "T-shirts" were uninspiring! At the end of each "show", we had a "thank you" to the actors for their incredible roles! And I must say, that for me, is NOT the greatest feature of the DVD, it would be much more enjoyable to just look through it. _SENTIMENTS_ positive _END_REVIEW_ In the early 1980's, one of my favorite films was "Willy Wonka and the Chocolate Factory." This film was based on The book of the same name by Rudyard Kipling and his brother Charles. In it, a couple of humans get stuck in a "gummy" (a "gulch" or a "cave" in this case) and have to come to a "magic" (a "factory" or a "dairy" in this case) to free themselves.<br /><br />In "Willy Wonka", there are 3 main characters, Jack, Willie, and Alice, who go to visit their friends on the other side of the "gummy". They get saved by a mysterious woman in black. The girls, 